## SELECTION DE 5 VARIABLES "ENTREPRISES" POUR LE DATASET FINAL ##

Dans notre dataset, nous disposons de 738 variables dont 168 concernant les entreprises réparties entre deux thématiques principales déclinées par plusieurs sous-thèmes:

- Démographie des entreprises
- Caractèristiques des établissements

Par entreprise, on distingue une [unité légale](https://www.insee.fr/fr/metadonnees/definition/c1044) d'un [établissement](https://www.insee.fr/fr/metadonnees/definition/c1377).

Les établissements étant beaucoup plus représentatifs du tissu économique que les unités légales, qui peuvent finalement n'être que des simples boites aux lettres ou des auto-entrepreneurs, nous choisissons volontairement d'orienter le choix des variables vers des données concernant les établissements.

Les 168 variables d'entreprises sont des déclinaisons des quantités d'entreprises sur une année par :
   - activité
   - taille
   - effectif
   - création
   - [sphère](https://www.insee.fr/fr/metadonnees/definition/c1886#:~:text=Il%20s'agit%20des%20activit%C3%A9s,les%20entreprises%20de%20cette%20sph%C3%A8re.)

A partir de ces variables, nous allons garder le nombre total d'entreprises par commune et créér 4 nouvelles variables :

- **ets_impact_local_p** : basé sur les données de sphère de l'INSEE, c'est le calcul du taux d'établissements ayant une sphère présentielle, c'est à dire ayant une une activité de production de biens et de services visant la satisfaction des besoins de personnes au niveeau local
- **ets_creation_p** : taux de création d'entreprises
- **ets_industrie_p** : taux d'établissements de type industrie
- **ets_taille50p_p** : taux d'établissements ayant 50 et plus salariés, la plus grande catégorie d'effectif proposée par l'INSEE

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('../all_content_2020_v4.csv', dtype={'CODGEO': 'string'})

C:\Users\vivie\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,8,519,520,521) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
ets = pd.DataFrame({'CODGEO': df.CODGEO, 
                    'ets_total':df.ETTOT20, 
                    'ets_local': df.ETPRES20, 
                    'ets_externe':df.ETNPRES20, 
                    'ets_creation': df.ETCTOT20,
                    'ent_creation': df.ENCTOT20,
                    'ets_industrie': df.ETBE20,
                    'ets_effectif50p_p': df.ETTEF5020})

In [4]:
aav = pd.read_csv('AAV20.csv', sep=';')

In [5]:
ets = ets.merge(aav, how='left', on='CODGEO')

In [6]:
ets.ets_total = ets.ets_total.replace(0.0, np.nan)

In [7]:
ets = ets.dropna(subset=['ets_total']).reset_index(drop=True)

In [8]:
ets.head()

,CODGEO,ets_total,ets_local,ets_externe,ets_creation,ent_creation,ets_industrie,ets_effectif50p_p,LIBGEO,AAV2020,LIBAAV2020,CATEAAV2020,DEP,REG
0,01001,14.0,9.0,5.0,12,11,0.0,0.0,L'Abergement-Clémenciat,524,Châtillon-sur-Chalaronne,20.0,01,84.0
1,01002,4.0,2.0,2.0,3,3,0.0,0.0,L'Abergement-de-Varey,000,Commune hors attraction des villes,30.0,01,84.0
2,01004,586.0,462.0,124.0,192,176,31.0,21.0,Ambérieu-en-Bugey,243,Ambérieu-en-Bugey,11.0,01,84.0
3,01005,46.0,36.0,10.0,24,23,4.0,0.0,Ambérieux-en-Dombes,002,Lyon,20.0,01,84.0
4,01006,2.0,1.0,1.0,0,0,0.0,0.0,Ambléon,286,Belley,20.0,01,84.0


In [9]:
ets['ets_impact'] = ets.apply(lambda x: 'local' if x['ets_local'] > x['ets_externe'] else ('externe' if x['ets_externe'] > x['ets_local'] else 'équilibré') , axis=1)

In [10]:
ets['ets_impact_local_p'] = ets.apply(lambda x:  np.round(x['ets_local'] / x['ets_total'] * 100,1), axis=1)

In [11]:
ets['ets_creation_p'] = ets.apply(lambda x: np.round(x['ets_creation'] / x['ets_total'] * 100,1), axis=1)

In [12]:
ets['ets_industrie_p'] = ets.apply(lambda x: np.round(x['ets_industrie'] / x['ets_total'] * 100,1), axis=1)

In [13]:
ets['ets_taille50p_p'] = ets.apply(lambda x: np.round(x['ets_effectif50p_p'] / x['ets_total'] * 100,1), axis=1)

In [14]:
ets.ets_impact.value_counts()

local        23339
externe       7919
équilibré     3572
Name: ets_impact, dtype: int64

In [15]:
#ets = ets.drop(['ets_local', 'ets_externe', 'ets_creation', 'ets_impact', 'ent_creation', 'ets_industrie', 'ets_effectif50p_p'], axis=1)

In [16]:
ets.head()

,CODGEO,ets_total,ets_local,ets_externe,ets_creation,ent_creation,ets_industrie,ets_effectif50p_p,LIBGEO,AAV2020,LIBAAV2020,CATEAAV2020,DEP,REG,ets_impact,ets_impact_local_p,ets_creation_p,ets_industrie_p,ets_taille50p_p
0,01001,14.0,9.0,5.0,12,11,0.0,0.0,L'Abergement-Clémenciat,524,Châtillon-sur-Chalaronne,20.0,01,84.0,local,64.3,85.7,0.0,0.0
1,01002,4.0,2.0,2.0,3,3,0.0,0.0,L'Abergement-de-Varey,000,Commune hors attraction des villes,30.0,01,84.0,équilibré,50.0,75.0,0.0,0.0
2,01004,586.0,462.0,124.0,192,176,31.0,21.0,Ambérieu-en-Bugey,243,Ambérieu-en-Bugey,11.0,01,84.0,local,78.8,32.8,5.3,3.6
3,01005,46.0,36.0,10.0,24,23,4.0,0.0,Ambérieux-en-Dombes,002,Lyon,20.0,01,84.0,local,78.3,52.2,8.7,0.0
4,01006,2.0,1.0,1.0,0,0,0.0,0.0,Ambléon,286,Belley,20.0,01,84.0,équilibré,50.0,0.0,0.0,0.0


In [17]:
ets.ets_industrie_p = ets.ets_industrie_p.fillna(0)
ets.ets_taille50p_p = ets.ets_taille50p_p.fillna(0)

In [18]:
np.average(ets['ets_creation_p'])

40.896017800746485

In [19]:
len(ets)

34830

In [20]:
ets.isna().sum()

CODGEO                0
ets_total             0
ets_local             0
ets_externe           0
ets_creation          0
ent_creation          0
ets_industrie         1
ets_effectif50p_p     1
LIBGEO                1
AAV2020               1
LIBAAV2020            1
CATEAAV2020           1
DEP                   1
REG                   1
ets_impact            0
ets_impact_local_p    0
ets_creation_p        0
ets_industrie_p       0
ets_taille50p_p       0
dtype: int64

In [21]:
ets.to_csv('variables_ets.csv', index=False)